# TensorFlow Tutorial

TensorFlow is an end-to-end platform for machine learning. It supports the following:

- Multidimensional-array based numeric computation (similar to NumPy.)
- GPU and distributed processing
- Automatic differentiation
- Model construction, training, and export

Keras is a high-level API for TensorFlow.
Keras provides default training and evaluation loops, `fit()` and `evaluate()`. Their usage is covered in the guide [Training & evaluation with the built-in methods](https://www.tensorflow.org/guide/keras/train_and_evaluate/).

If you want to customize the learning algorithm of your model while still leveraging the convenience of `fit()`, you can subclass the `Model` class and implement your own `train_step()` method, which is called repeatedly during `fit()`.

Here instead we will write our own training & evaluation loops from scratch.

## Setup

In [1]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

import time

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


## Get the data
Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/).

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

Use `tf.data` to batch and shuffle the dataset:

In [3]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

## Model

Build the `tf.keras` model using the Keras [model subclassing API](https://www.tensorflow.org/guide/keras#model_subclassing):

In [4]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# Create an instance of the model
model = MyModel()

Choose an optimizer and loss function for training: 

In [5]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

Select metrics to measure the loss and the accuracy of the model. These metrics accumulate the values over epochs and then print the overall result.

In [6]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## Eager Execution

The `train_step` function is called at each epoch and needs to compute the gradients for backpropagation.<br/>
Use `tf.GradientTape` to record the operations run
during the forward pass, which enables auto-differentiation.

In [7]:
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # Run the forward pass of the model.
    # The operations that the model applies to its inputs
    # are being recorded on the GradientTape.

    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_fn(labels, predictions)
  # Use the gradient tape to automatically retrieve
  # the gradients of the trainable variables with respect to the loss.
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss.update_state(loss)
  train_accuracy.update_state(labels, predictions)

The `test_step` function is used for evaluating the model after each epoch:

In [8]:
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_fn(labels, predictions)

  test_loss.update_state(t_loss)
  test_accuracy.update_state(labels, predictions)

Here's our training loop:

- We open a for loop that iterates over epochs
- For each epoch, we open a for loop that iterates over the dataset, in batches
- For each batch, we open a GradientTape() scope
- Inside this scope, we call the model (forward pass) and compute the loss
- Outside the scope, we retrieve the gradients of the weights of the model with regard to the loss
- Finally, we use the optimizer to update the weights of the model based on the gradients

In [9]:
EPOCHS = 2

for epoch in range(EPOCHS):
  start_time = time.time()

  # Iterate over the batches of the dataset.
  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
    f'Epoch {epoch + 1}, Time: {(time.time() - start_time):.2f}\n'
    f'Loss: {train_loss.result():.4f}, '
    f'Accuracy: {train_accuracy.result() * 100:.2f}%, '
    f'Test Loss: {test_loss.result():.4f}, '
    f'Test Accuracy: {test_accuracy.result() * 100:.2f}%'
  )

  # Reset losses and metrics at the end of each epoch
  train_loss.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()
  train_accuracy.reset_states()


Epoch 1, Time: 50.57
Loss: 0.1440, Accuracy: 95.69%, Test Loss: 0.0656, Test Accuracy: 97.96%
Epoch 2, Time: 50.05
Loss: 0.0450, Accuracy: 98.60%, Test Loss: 0.0514, Test Accuracy: 98.22%


## Graph Execution

For Graph execution, we turn the `train_step` and `test_step` into graph functions, by decorating the definition with `@tf.function`:

In [10]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_fn(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss.update_state(loss)
  train_accuracy.update_state(labels, predictions)

Same for the evaluation function:

In [11]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_fn(labels, predictions)

  test_loss.update_state(t_loss)
  test_accuracy.update_state(labels, predictions)

Training loop with the compiled training and test step functions:

In [12]:
EPOCHS = 2

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()
  start_time = time.time()

  # Iterate over the batches of the dataset.
  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
   f'Epoch {epoch + 1}, Time: {(time.time() - start_time):.2f}\n'
    f'Loss: {train_loss.result():.4f}, '
    f'Accuracy: {train_accuracy.result() * 100:.2f}%, '
    f'Test Loss: {test_loss.result():.4f}, '
    f'Test Accuracy: {test_accuracy.result() * 100:.2f}%'
  )

Epoch 1, Time: 26.52
Loss: 0.0250, Accuracy: 99.13%, Test Loss: 0.0513, Test Accuracy: 98.51%
Epoch 2, Time: 26.08
Loss: 0.0148, Accuracy: 99.51%, Test Loss: 0.0595, Test Accuracy: 98.28%


The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials).

# Graphs

In [ ]:
def d():
    return tf.add(3, 5)

tf_d = tf.function(d)

In [18]:
tf.add(3, 5)

<tf.Tensor: shape=(), dtype=int32, numpy=8>

In [25]:
tf_d.get_concrete_function().graph.as_graph_def()

node {
  name: "Add/x"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
        }
        int_val: 3
      }
    }
  }
}
node {
  name: "Add/y"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
        }
        int_val: 5
      }
    }
  }
}
node {
  name: "Add"
  op: "AddV2"
  input: "Add/x"
  input: "Add/y"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
node {
  name: "Identity"
  op: "Identity"
  input: "Add"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
versions {
  producer: 987
}

# Gradients

In [26]:
def square(x):
    return tf.multiply(x, x)

square(3.).numpy()   # x^2 = 9.

9.0

In [27]:
def dsquare(x):
   with tf.GradientTape() as t:
    return t.gradient(square(x), x)

x = tf.Variable(3.)
dsquare(x).numpy()   # 2 * x = 6.

6.0

# TensorBoard

In [37]:
%load_ext tensorboard
import datetime

In [29]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=3, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

Epoch 1/3
1875/1875 [==============================] - 27s 14ms/step - loss: 0.8507 - accuracy: 0.9373 - val_loss: 1.2059 - val_accuracy: 0.7958
Epoch 2/3
1875/1875 [==============================] - 26s 14ms/step - loss: 1.5824 - accuracy: 0.9028 - val_loss: 1.6729 - val_accuracy: 0.9061
Epoch 3/3
1875/1875 [==============================] - 26s 14ms/step - loss: 1.9408 - accuracy: 0.8047 - val_loss: 2.1050 - val_accuracy: 0.6502


In [38]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 3204795), started 0:09:17 ago. (Use '!kill 3204795' to kill it.)